In [1]:
# 获取2021-2024PlayersSalaries.csv的编码格式
import chardet
with open('2021-2024PlayersSalaries.csv', 'rb') as f:
    result = chardet.detect(f.read())
print(result['encoding'])

UTF-8-SIG


In [2]:
# 获取2023-2024 NBA Player Stats - Regular.csv的编码格式
import chardet
with open('2023-2024 NBA Player Stats - Regular.csv', 'rb') as f:
    result = chardet.detect(f.read())
print(result['encoding'])

Windows-1252


In [ ]:
#把2023-2024 NBA Player Stats - Regular.csv转换成utf-8编码，并另存
import codecs
# 读取文件
with open('2023-2024 NBA Player Stats - Regular.csv', 'r', encoding='Windows-1252') as source_file:
    content = source_file.read()
# 写入新文件，编码为 UTF-8
with open('2023-2024 NBA Player Stats - Regular_utf8.csv', 'w', encoding='utf-8') as target_file:
    target_file.write(content)

In [ ]:
# 数据基本情况展示
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 导入数据
player_stats = pd.read_csv('2023-2024 NBA Player Stats - Regular_utf8.csv')
player_salaries = pd.read_csv('2021-2024PlayersSalaries.csv')


# 显示数据集基本情况
print("球员赛场表现数据集的前 5 行：")
print(player_stats.head())
print("\n球员薪资数据集的前 5 行：")
print(player_salaries.head())

# 数据集基本信息
print("\n球员赛场表现数据集的基本信息：")
print(player_stats.info())
print("\n球员薪资数据集的基本信息：")
print(player_salaries.info())

# 数据集描述性统计
print("\n球员赛场表现数据集的描述性统计：")
print(player_stats.describe())
print("\n球员薪资数据集的描述性统计：")
print(player_salaries.describe())

In [ ]:
# 数据预处理、异常值处理、缺失值处理
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 加载数据
stats_df = pd.read_csv('2023-2024 NBA Player Stats - Regular_utf8.csv')
salaries_df = pd.read_csv('2021-2024PlayersSalaries.csv')

# 数据预处理 - 只保留2023-2024赛季的薪资数据
salaries_2023_2024 = salaries_df[['Player', 'Salary(2023-2024)']]

# 数据合并
merged_df = pd.merge(stats_df, salaries_2023_2024, on='Player', how='inner')

# 转换薪资数据类型为float
merged_df['Salary(2023-2024)'] = merged_df['Salary(2023-2024)'].astype(float)

plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体为黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# =============================================================================
# 数据集的基本情况展示
# =============================================================================
# 显示数据集基本情况
print("球员赛场表现数据集的前 5 行：")
print(stats_df.head())
print("\n球员薪资数据集的前 5 行：")
print(salaries_2023_2024.head())
# 查看数据集的基本信息
print("\n=== 数据集基本情况 ===")
print(f"数据集的形状：{merged_df.shape}")  # 输出数据集的行数和列数
print(f"各列的数据类型：\n{merged_df.dtypes}")  # 输出各列的数据类型
print(f"数据集的描述性统计信息：\n{merged_df.describe(include='all')}")  # 输出数据集的描述性统计信息
print(f"数据集的前5行：\n{merged_df.head()}")  # 输出数据集的前5行
print(f"数据集的后5行：\n{merged_df.tail()}")  # 输出数据集的后5行

# 数据集的描述性统计信息
desc_stats = merged_df.describe().transpose()

# 绘制 count 的柱状图
plt.figure(figsize=(12, 6))
desc_stats['count'].plot(kind='bar')
plt.title('数据集各列的样本数量', fontsize=14)
plt.xlabel('列名', fontsize=12)
plt.ylabel('样本数量', fontsize=12)
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# =============================================================================
# 缺失值处理方法
# =============================================================================
# 检查数据集中的缺失值情况
print("\n=== 缺失值处理 ===")
print(f"数据集中各列的缺失值情况：\n{merged_df.isnull().sum()}")  # 输出各列的缺失值数量

# 对于数值型列，使用该列的均值进行填充
numeric_columns = merged_df.select_dtypes(include=['float64', 'int64']).columns
for column in numeric_columns:
    if merged_df[column].isnull().any():
        merged_df[column].fillna(merged_df[column].mean(), inplace=True)
        print(f"对数值型列 '{column}' 使用均值填充了 {merged_df[column].isnull().sum()} 个缺失值")

# 对于字符型列，使用该列的众数进行填充
categorical_columns = merged_df.select_dtypes(include=['object']).columns
for column in categorical_columns:
    if merged_df[column].isnull().any():
        mode_value = merged_df[column].mode()[0]
        merged_df[column].fillna(mode_value, inplace=True)
        print(f"对字符型列 '{column}' 使用众数 '{mode_value}' 填充了 {merged_df[column].isnull().sum()} 个缺失值")

print("\n处理后的缺失值情况：")
print(merged_df.isnull().sum())  # 再次检查缺失值情况

# =============================================================================
# 异常值处理方法
# =============================================================================
# 使用箱线图可视化数值型列的异常值
print("\n=== 异常值处理 ===")
plt.figure(figsize=(15, 10))
merged_df[numeric_columns].boxplot()
plt.title('数值型列的箱线图（异常值可视化）')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# 使用IQR方法处理数值型列的异常值
for column in numeric_columns:
    # 计算四分位数
    Q1 = merged_df[column].quantile(0.25)
    Q3 = merged_df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    # 定义异常值的上下界
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # 替换异常值为上下界
    merged_df[column] = np.where(merged_df[column] < lower_bound, lower_bound, merged_df[column])
    merged_df[column] = np.where(merged_df[column] > upper_bound, upper_bound, merged_df[column])

print(f"异常值处理后的数据集形状：{merged_df.shape}")

# 再次使用箱线图可视化数值型列的异常值处理结果
plt.figure(figsize=(15, 10))
merged_df[numeric_columns].boxplot()
plt.title('数值型列的箱线图（异常值处理后）')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# 选择数值型列
numeric_columns = merged_df.select_dtypes(include=['float64', 'int64']).columns

# 计算数值型列的相关系数矩阵
correlation_matrix = merged_df[numeric_columns].corr()

# 提取薪资与各表现指标的相关系数
salary_correlations = correlation_matrix['Salary(2023-2024)'].sort_values(ascending=False)

# 打印薪资与各表现指标的相关系数
print("\n=== 薪资与各表现指标的皮尔逊相关系数 ===")
print(salary_correlations)

# 可视化薪资与各表现指标的相关性（热力图）
# 优化热力图显示效果
plt.figure(figsize=(18, 14))  # 增大图形尺寸以提供更多的空间
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', center=0, square=True,
            annot_kws={"size": 8}, cbar_kws={"shrink": 0.8, "label": "相关系数"})

# 调整标题和标签的字体大小
plt.title('球员赛场表现与薪资之间的皮尔逊相关系数热力图', fontsize=18, pad=20)
plt.xlabel('变量', fontsize=14, labelpad=15)
plt.ylabel('变量', fontsize=14, labelpad=15)
plt.xticks(fontsize=10, rotation=45, ha='right')  # 调整X轴标签字体大小和旋转角度
plt.yticks(fontsize=10, rotation=0)  # 调整Y轴标签字体大小，保持垂直显示

# 添加边距以防止标签被截断
plt.tight_layout()
plt.subplots_adjust(top=0.95, bottom=0.1, left=0.1, right=0.95)
plt.show()

# 可视化薪资与得分（PTS）的关系
plt.figure(figsize=(12, 6))
plt.scatter(merged_df['PTS'], merged_df['Salary(2023-2024)'], alpha=0.6)
plt.title('得分（PTS）与薪资（Salary）的关系')
plt.xlabel('得分（PTS）')
plt.ylabel('薪资（Salary）')
plt.tight_layout()
plt.show()

# 可视化薪资与投篮命中数（FG）的关系
plt.figure(figsize=(12, 6))
plt.scatter(merged_df['FG'], merged_df['Salary(2023-2024)'], alpha=0.6, color='blue')
plt.title('投篮命中数（FG）与薪资（Salary）的关系')
plt.xlabel('投篮命中数（FG）')
plt.ylabel('薪资（Salary）')
plt.tight_layout()
plt.show()

# 可视化薪资与投篮出手数（FGA）的关系
plt.figure(figsize=(12, 6))
plt.scatter(merged_df['FGA'], merged_df['Salary(2023-2024)'], alpha=0.6, color='green')
plt.title('投篮出手数（FGA）与薪资（Salary）的关系')
plt.xlabel('投篮出手数（FGA）')
plt.ylabel('薪资（Salary）')
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 过滤掉具有多个位置的球员
single_pos_players = merged_df[merged_df['Pos'].isin(['C', 'SF', 'PF', 'SG', 'PG'])]

# 按位置分组计算相关系数
pos_grouping = single_pos_players.groupby("Pos")
positions = single_pos_players["Pos"].unique()

# 创建一个字典来存储每个位置的相关系数
pos_correlation = {}

for pos in positions:
    pos_df = pos_grouping.get_group(pos)
    numeric_columns = pos_df.select_dtypes(include=['float64', 'int64']).columns
    correlation_matrix = pos_df[numeric_columns].corr()
    pos_correlation[pos] = correlation_matrix['Salary(2023-2024)']

# 打印每个位置的薪资与各表现指标的相关系数
for pos in positions:
    print(f"\n=== 位置：{pos} - 薪资与各表现指标的皮尔逊相关系数 ===")
    print(pos_correlation[pos].sort_values(ascending=False))

# 绘制每个位置的相关系数热力图
for pos in positions:
    plt.figure(figsize=(12, 8))
    sns.heatmap(pos_correlation[pos].to_frame().T, annot=True, cmap='coolwarm', center=0)
    plt.title(f'位置：{pos} - 表现指标与薪资之间的皮尔逊相关系数')
    plt.xlabel('表现指标')
    plt.ylabel('')
    plt.tight_layout()
    plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 过滤出具有单一位置的球员（C、SF、PF、SG、PG）
single_pos_players = merged_df[merged_df['Pos'].isin(['C', 'SF', 'PF', 'SG', 'PG'])]

# 按位置分组
pos_grouping = single_pos_players.groupby("Pos")

# 定义各位置与薪资最相关的指标
pos_specific_indicators = {
    'C': ['2P', 'DRB', 'TRB', 'BLK'],
    'SG': ['MP', 'AST', 'PF', '3PA'],
    'PF': ['AST', 'TOV', 'GS', 'DRB'],
    'PG': ['MP', 'DRB', 'TRB', '3P'],
    'SF': ['2PA', 'AST', 'GS', 'FTA']
}

# 可视化每个位置的相关指标与薪资的关系
for pos in pos_specific_indicators:
    pos_df = pos_grouping.get_group(pos)
    indicators = pos_specific_indicators[pos]
    
    # 创建一个包含3x2子图的画布
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'位置：{pos} - 与薪资最相关的指标', fontsize=16)
    
    # 为每个指标创建散点图
    for i, indicator in enumerate(indicators):
        row = i // 2
        col = i % 2
        ax = axes[row, col]
        
        # 绘制散点图
        ax.scatter(pos_df[indicator], pos_df['Salary(2023-2024)'], alpha=0.6)
        ax.set_title(f'{indicator} 与薪资的关系')
        ax.set_xlabel(indicator)
        ax.set_ylabel('薪资（Salary）')
        
        # 计算并显示皮尔逊相关系数
        correlation = pos_df[[indicator, 'Salary(2023-2024)']].corr().iloc[0, 1]
        ax.text(0.05, 0.95, f'r={correlation:.2f}', transform=ax.transAxes, fontsize=10)
    
    # 调整布局
    plt.tight_layout(rect=[0, 0, 1, 0.96])  # 为总标题留出空间
    plt.show()